# Group Mahalanobis Project 1


## Introduction/Brief Description

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nam pulvinar sagittis erat, eget commodo lectus luctus in. Sed a erat eget felis convallis ultricies. Morbi pharetra quam at volutpat pharetra. Donec bibendum ex sed dignissim imperdiet. Proin fringilla purus tellus, vel posuere eros sagittis vitae. Cras commodo suscipit dui, quis commodo ex pharetra sed. Orci varius natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Ut elit massa, dignissim et dui at, molestie hendrerit dui. Nunc non blandit tellus.


## Project Deliverables

### [#A] Data Imports

In [1]:
#If necessary, uncomment and run lines below:
#!pip install wbdata
#!pip install cufflinks

import wbdata
import cufflinks as cf
import pandas as pd
cf.go_offline()

### [#A] Population Statistics

A python function named population
that will deliver information to
answer the queries of the following
form:

In [year] how many
[people/males/females] aged [low] to
[high] were living in [the
world/region/country]?

WBDATA API ref: https://wbdata.readthedocs.io/en/stable/wbdata_library.html

In [2]:
"""
Parameters
----------------
years: 1960-2020
group: 'people', 'females', 'males'
age_lower: lower bound of age group rounded down to nearest 5
age_higher: lower bound of age group rounded up to nearest 5
location: WLD (world)/country 3-letter codes

population ranges available from dataset:
00-04
05-09
10-14
15-19
20-24
25-29
30-34
35-39
40-44
45-49
50-54
55-59
60-64
65-69
70-74
75-79
above 80
"""

def population(year, group, age_lower, age_upper, location):
    #female + male
    if group == 'people':
        return none
    #female/male
    else:
        return population_helper(year, group, age_lower, age_upper, location)


# Returns a list of input strings for population age ranges
#indicator format: e.g. SP.POP.0004.FE.5Y
def population_helper(year, group, age_lower, age_upper, location):
    inputs = list_of_age_inputs(age_lower, age_upper)
    args_to_indi = {
        "males": "MA",
        "females": "FE",
    }
      
    #create indicator dictionary
    indicator_dict = {}
    for i in inputs:
        key = "SP.POP.{}.{}.5Y".format(i, (args_to_indi[group]))
        value = "Age {} to {}".format(i[:2], i[2:])
        indicator_dict[key] = value
    df = wbdata.get_dataframe(indicator_dict, country=location)
    return df

# Returns a list of input strings for population age ranges
def list_of_age_inputs(age_lower, age_upper):
    def round_down(n):
        return n - n % 5
    
    def round_up(n):
        return n - n % 5 + 5
    
    results = []
    r_lower_bound, r_upper_bound = round_down(age_lower), round_up(age_upper)   
    while (r_lower_bound < min(79, r_upper_bound)):
        results.append("{:02d}{:02d}".format(r_lower_bound, r_lower_bound + 4))
        r_lower_bound += 5       
    return results

population(2020, 'males', 0, 50, 'USA')
    

,Age 00 to 04,Age 05 to 09,Age 10 to 14,Age 15 to 19,Age 20 to 24,Age 25 to 29,Age 30 to 34,Age 35 to 39,Age 40 to 44,Age 45 to 49,Age 50 to 54
date,,,,,,,,,,,
2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019,6.153330,6.369443,6.636408,6.655678,7.015787,7.433873,7.070153,6.554700,6.165810,6.159868,6.430893
2018,6.162448,6.501650,6.661853,6.699963,7.139412,7.413870,6.995028,6.486387,6.174732,6.250694,6.574907
2017,6.190469,6.622880,6.670024,6.751103,7.268989,7.364122,6.920691,6.428542,6.204453,6.358585,6.723264
2016,6.269949,6.712602,6.674819,6.813085,7.375933,7.300735,6.845917,6.384887,6.252850,6.482022,6.860353
...,...,...,...,...,...,...,...,...,...,...,...
1964,10.542620,10.813815,9.970752,8.673193,6.799892,5.977102,6.046372,6.399562,6.448021,5.876202,5.413588
1963,10.882823,10.778797,9.894824,8.388771,6.613361,5.966404,6.173400,6.549346,6.445949,5.891580,5.419146
1962,11.183908,10.721493,9.813903,8.079322,6.447545,6.004832,6.323720,6.693171,6.424417,5.925501,5.418200


### [#A] Population DataFrames

A function that returns a pandas DataFrame
indexed by Region or Country and Year, with
columns giving counts of people in different
age-sex groups.

### [#B] Population Pyramids

A python function that takes as
input a pandas DataFrame with
columns providing counts of people by
age-sex groups, and constructs a
“population pyramid” graph for
visualizing the data.

In [3]:
1+1

2

### [#C] Animated Population Pyramids

A python function that takes as
input a pandas DataFrame with
columns providing counts of people by
age-sex groups, with rows
corresponding to different years, and
constructs an animated “population
pyramid” graph for visualizing how the
population changes over time.

### [#C] Population Maps

A Jupyter notebook which queries a
population database and provides
visualizations in the form of maps.
An example of a library that may be
useful is geopandas.

### [#C] Other Visualization Tools

If you think of other ways to organize
or visualize population and how it
changes over time, this is for you!

### [#C] Agricultural Supply

Relate changes in agricultural food supply
to changes in population.

What would Malthus think?

## Skills

[#A] Jupyter Notebooks
Basic manipulations of Jupyter notebooks
on https://datahub.berkeley.edu

[#A] Use of Indicators API from python
Use the Indicators API to query the
on-line World Development Indicators
database.

[#A] Build pandas.DataFrame
Collect data on population into a
pandas DataFrame.

[#B] Read on-line spreadsheets using python
[#B] Construct plots in python
Using matplotlib, plotly, or similar.

[#C] Construct world maps
Consider using plotly, geopandas, or similar.

## Further/Others